# CS 182: Mike

Markov model, Beam search w/NB classifier as heuristic, Markovify (https://github.com/jsvine/markovify)

In [1]:
%matplotlib inline
import datetime
import json
import random
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import re
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('poster')
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)
warnings.simplefilter(action = 'ignore', category = DeprecationWarning)

In [2]:
# import data from augmented.csv
filename = 'augmented.csv'
augmented_df = pd.read_csv(filename)
gh_data = augmented_df[augmented_df['clickbait'] == 1]

print 'There are', len(gh_data), 'clickbait articles.'
print gh_data.head()

There are 4643 clickbait articles.
   Unnamed: 0                                      article_title                                        article_url  clickbait    source
0           0             23 Life Lessons Cosmo Kramer Taught Us  /javiermoreno/ife-lessons-you-learned-from-cos...          1  Buzzfeed
1           1          32 Men On TV Who Made You Thirsty In 2014  /erinlarosa/32-tv-men-who-made-you-thirsty-in-...          1  Buzzfeed
2           2          Hilary Duff Was The Walking Queen Of 2014  /lyapalater/hilary-duff-was-the-walking-queen-...          1  Buzzfeed
3           3        25 Reasons Wine Is Definitely Your Soulmate  /emleschh/25-reasons-why-wine-is-your-soulmate...          1  Buzzfeed
4           4  This Master Carver Making Pliers From One Stic...          /norbertobriceno/ernest-macguyver-warther          1  Buzzfeed


In [3]:
### Sanitize titles for Markov model, output each token on separate line in article_titles.txt ###

def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

def removeQuotations(s):
    counter = 0
    for i in s:
        if i == '\"':
            counter += 1
    return not (counter == 2 or counter == 0)

def removeParens(s):
    counter = 0
    for i in s:
        if i == '(' or i == ')':
            counter += 1
    return not (counter == 2 or counter == 0)

titles = open('article_titles.txt', 'w')
counter = 0
for i in gh_data['article_title']:
    nonAscii = removeNonAscii(i)
    strlist = list(nonAscii)
    lastchar = strlist[len(strlist) - 1]
    if lastchar != '.' and lastchar != '!' and lastchar != '?':
        strlist.append('.')
    nonAscii = ''.join(strlist)
    tokens = nonAscii.split()
    if len(tokens) < 4:
        continue
    for token in tokens:
        if removeQuotations(token):
            token = token.replace('\"', "")
        if removeParens(token):
            token = token.replace('(', "")
            token = token.replace(')', "")
        if token.isupper():
            if len(token) != 1 or token[0] == 'A':
                token = token.lower()
                tokenlist = list(token)
                tokenlist[0] = tokenlist[0].upper()
                token = "".join(tokenlist)
        tokenlist = list(token)
        tokenlist.append('\n')
        titles.write(''.join(tokenlist))
titles.close()

In [4]:
### Construct word-to-int mapping, set of existing titles for comparison below.  ###
### Also this is kinda dumb but markovify expects 1 line == 1 sentence so write to markovify_titles.txt ###

words = open('article_titles.txt', 'r')
fullTitles = open('markovify_titles.txt', 'w')

wordMap = {}
reverseWordMap = {}
existingTitles = []
currentTitle = []
totalWords = 0
for line in words:
    # Construct mapping of words to ints
    line = line.strip()
    if line == '':
        continue
    if line not in wordMap:
        wordMap[line] = len(wordMap)
        reverseWordMap[wordMap[line]] = line
    
    # Construct set of article titles to test for similarity
    currentTitle.append(wordMap[line])
    lastchar = line[len(line)-1]
    if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
        existingTitles.append(currentTitle)
        totalWords += len(currentTitle)
        fullTitles.write(" ".join(map(lambda x: reverseWordMap[x], currentTitle) + ['\n']))
        currentTitle = []
        
words.close()
fullTitles.close()

print totalWords
print len(existingTitles)

53897
5705


In [5]:
### Construct transition matrix of words, word frequency (mapping from word to count), and starter word frequency ###

words = open('article_titles.txt', 'r')
transitionMatrix = {}
wordFrequency = {}
beginnerFrequency = {}

prevToken = None
nextWordStart = True
for line in words:
    line = line.strip()
    if line == '':
        continue
    
    if line not in wordFrequency:
        wordFrequency[line] = 0
    wordFrequency[line] += 1
    
    if nextWordStart:
        if line not in beginnerFrequency:
            beginnerFrequency[line] = 0
        beginnerFrequency[line] += 1
        nextWordStart = False
    
    if prevToken == None:
        prevToken = line
    else:
        if prevToken not in transitionMatrix:
            transitionMatrix[prevToken] = {}
        if line not in transitionMatrix[prevToken]:
            transitionMatrix[prevToken][line] = 0
        transitionMatrix[prevToken][line] += 1
    
    lastchar = line[len(line)-1]
    if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
        prevToken = None
        nextWordStart = True
    else:
        prevToken = line

In [6]:
### Helper functions for Markov model ###

# Construct successor probability dist based on global word frequency
def constructDist(keys):
    frequencies = []
    for key in keys:
        frequencies.append(wordFrequency[key])
    return map(lambda x: x / float(sum(frequencies)), frequencies)

# Construct successor probability dist based on local word frequency
def constructLocalDist(keys, frequencyTable):
    frequencies = []
    for key in keys:
        frequencies.append(frequencyTable[key])
    return map(lambda x: x / float(sum(frequencies)), frequencies)

# Find fraction of title2 words appear in title1 (i.e. compare bags of words)
def compareWords(title1, title2):
    counter = 0
    for i in title2:
        if i in title1:
            counter += 1
    return float(counter) / len(title2)

# Check for similarity against existing titles, return score of "best" (as in, most similar) existing title
def checkTable(phrase):
    # generate the list
    newPhrase = []
    tokens = phrase
    for token in tokens:
        newPhrase.append(wordMap[token])
    
    # Check against all preexisting titles
    bestScore = 0
    mostSimilar = None
    for title in existingTitles:
        score = compareWords(title, newPhrase)
        if score > bestScore:
            bestScore = score
            mostSimilar = title
    
    return bestScore

In [35]:
### Random walk through transition matrix, start from a starter word and end at punctuation (., !, ?)

# tweakable parameters

# min/max numbers of tokens per title
minLength = 5
maxLength = 15

# number of titles to generate
numTitles = 50

# maximum allowed proportion of shared words b/t generated and preexisting title
similarityThreshold = 0.5

# don't touch these
phrase = []
prevWord = None
counter = 0
while True:
    while True:
        if prevWord == None:
                keys = beginnerFrequency.keys()
                word = np.random.choice(keys, 1, constructLocalDist(keys, beginnerFrequency))
                phrase.append(word[0])
                prevWord = word[0]
        else:
            if prevWord not in transitionMatrix:
                phrase = []
                prevWord = None
            else:
                keys = transitionMatrix[prevWord].keys()
                word = np.random.choice(keys, 1, constructLocalDist(keys, transitionMatrix[prevWord]))
                prevWord = word[0]
                phrase.append(word[0])
                lastchar = word[0][len(word[0])-1]
                if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
                    break
    if len(phrase) < minLength or len(phrase) > maxLength:
        phrase = []
        prevWord = None
    elif checkTable(phrase) > similarityThreshold:
        phrase = []
        prevWord = None
    else:
        counter += 1
        print " ".join(phrase)
        phrase = []
        prevWord = None
    if counter > numTitles - 1:
        break

Nothing But That Even Think They're Still Adorable.
'Dynasty' Star Allison Williams Angers Fans Reportedly Was Rear-Ended Him...Then They Won.
Katherine Webb Is Breaking a Salty Snack.
Katy Perry Launches Own Existence...I Am Alive .
Determined To Realize I've Ever To Hack: 11 Times Starbucks Failed To Them.
'Glee' Departure, Says They Wrote Notes That Scout Balances On Bid Day.
Partner Brittany Murphy's Final Rose': Nick Carter Marries Lauren Kitt.
Leonardo DiCaprio Felt like Nick Claims Yale Threatened Forced To Acting...And These Trains.
Arnold Schwarzenegger How Bad For Men Alive Around Us .
Didn't Even Seem Boring....But When I Would Love It.
People Need No Longer Attending Her Husband.
Oscar-Nominated Filmmaker Making Pliers From Complications With Knowledge.
Rowling's 'The Bachelorette', Calls Andi Dorfman & Its Alibaba Winnings To Mars.
Met Until This Painter's Breathtaking Photographs That Came To Angsty Teenagers.
X-Men: Days Are Now is Hotttt!
'Pretty Little Kittens Who Just

In [46]:
### Initialize Naive Bayes model ###

from sklearn.feature_extraction.text import TfidfVectorizer
from naive_bayes import NaiveBayes

filename = 'augmented.csv'
augmented_df = pd.read_csv(filename)

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
X = vectorizer.fit_transform(augmented_df['article_title'])
y = np.array(augmented_df['clickbait'])

nb_madhu = NaiveBayes()
nb_madhu.fit(X, y, vectorizer.vocabulary_, alpha=0.5)

In [ ]:
%%time
### Beam search w/NB heuristic ###

import operator
from copy import deepcopy
from math import exp

# tweakable parameters
minLength = 5
maxLength = 15
numTitles = 20
beamWidth = 50

# Construct successor probability distribution based on NB probability 
def constructBayesDist(phrase, keys):
    probs = []
    for key in keys:
        newPhrase = phrase + [key]
        result = nb_madhu.predict_proba(vectorizer.transform([" ".join(newPhrase)]))
        prob = result[0][1]
        probs.append(prob)
    # randomly weight each probability by e^(-.5x), where 0 <= x <= 1
    probs = map(lambda x: x * exp(-.5 * random.random()), probs)
    return probs

prevNext = []
currNext = []
finishedCandidates = []
firstLevel = True
terminateMe = False

while not terminateMe:
    if firstLevel == True:
        keys = beginnerFrequency.keys()
        keyValList = zip([[] for i in range(len(keys))], keys, constructBayesDist([], keys))
        keyValList.sort(key=operator.itemgetter(2), reverse=True)
        newBeamWidth = min(len(keyValList), beamWidth)
        for i in range(newBeamWidth):
            prevNext.append(keyValList[i])
        firstLevel = False
    else:
        for prefix, newword, prob in prevNext:
            if len(prefix) >= maxLength-1:
                terminateMe = True
                break
            if newword not in transitionMatrix:
                # this only happens when we randomly select an end word at the start
                continue
            keys = transitionMatrix[newword].keys()
            keyValList = zip([prefix + [newword] for i in range(len(keys))], keys, constructBayesDist(prefix, keys))
            keyValList.sort(key=operator.itemgetter(2), reverse=True)
            newBeamWidth = min(len(keyValList), beamWidth)
            for i in range(newBeamWidth):
                lastchar = keyValList[i][1][len(keyValList[i][1])-1]
                if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
                    finishedCandidates.append(keyValList[i])
                else:
                    currNext.append(keyValList[i])
                    
        # sort results of the next depth and remove elements outside the beam
        currNext.sort(key=operator.itemgetter(2), reverse=True)
        if len(currNext) > beamWidth:
            del currNext[beamWidth:]

        prevNext = deepcopy(currNext)
        currNext = []

# sort finished candidates by clickbait probability and output the results
finishedCandidates = filter(lambda x: len(x[0]) >= minLength, finishedCandidates)
finishedCandidates.sort(key=operator.itemgetter(2), reverse=True)
newBeamWidth = min(len(finishedCandidates), numTitles)
for i in range(newBeamWidth):
    print " ".join(finishedCandidates[i][0] + [finishedCandidates[i][1]]) 

In [108]:
### Use markovify library https://github.com/jsvine/markovify ###

import markovify

with open("markovify_titles.txt") as f:
    text = f.read()

# state_size=3 results look cleaner, but are less novel and more likely to closely resemble a headline in the corpus.
text_model = markovify.Text(text, state_size=2)

for i in range(5):
    print(text_model.make_sentence())

11 Cakes That Look . The Kitten Soldiers Of World War Ii Ghost Photographs.
This Is What It's Really Shocking.
23 Times Louis Tomlinson Smoking a Joint.
Chris Hemsworth and Elsa Pataky Giving Birth Over And Over Without Stopping Since Last Month!
13 Pics of Little Kids Who Dress Better Than The Average Biopic.
